# PySpark Environment

In [1]:
# Set the pyspark environment
# Use the conda environment in this path: /Users/rmontecino/anaconda3/envs/pyspark-env
import os
os.environ['SPARK_HOME'] = '/Users/rmontecino/anaconda3/envs/pyspark-env/lib/python3.12/site-packages/pyspark'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'
os.environ['PYSPARK_PYTHON'] = 'python3'

In [2]:
# Import PySpark and initialize SparkSession
from pyspark.sql import SparkSession

In [3]:
# Create a SparkSession
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

24/08/07 21:23:19 WARN Utils: Your hostname, Ricardos-MacBook-Pro-CleverIT.local resolves to a loopback address: 127.0.0.1; using 192.168.1.86 instead (on interface en0)
24/08/07 21:23:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 21:23:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 21:23:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/07 21:23:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53424)
Traceback (most recent call last):
  File "/Users/rmontecino/anaconda3/envs/pyspark-env/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/rmontecino/anaconda3/envs/pyspark-env/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/Users/rmontecino/anaconda3/envs/pyspark-env/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/rmontecino/anaconda3/envs/pyspark-env/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/Users/rmontecino/anaconda3/envs/pyspark-env/lib/python3.12/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/rmontecino/anaconda3/envs/pyspark-env/lib/pyth

# Load Data into a DataFrame

In [4]:
# Load the synthetic data into a DataFrame
data_file = '../data/persons.csv'
persons_df = spark.read.csv(data_file, header=True, inferSchema=True)

# Show the schema of the DataFrame
persons_df.printSchema()

# Show the first 5 rows of the DataFrame
persons_df.show(5)

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------+---+------+------+
|          name|age|gender|salary|
+--------------+---+------+------+
|      John Doe| 30|  Male| 50000|
|    Jane Smith| 25|Female| 45000|
| David Johnson| 35|  Male| 60000|
|   Emily Davis| 28|Female| 52000|
|Michael Wilson| 40|  Male| 75000|
+--------------+---+------+------+
only showing top 5 rows



# Register the DataFrame as a Temporary Table

In [5]:
# Register the DataFrame as a Temporary Table
persons_df.createOrReplaceTempView('persons')

# Perform SQL-like Queries

In [6]:
# Select all rows where age is greater than 25
query = 'SELECT * FROM persons WHERE age > 25'
persons_df_greater_than_25 = spark.sql(query)
persons_df_greater_than_25.show()

# Compute the average salary of persons
query = 'SELECT AVG(salary) AS avg_salary FROM persons'
avg_salary = spark.sql(query)
avg_salary.show()

+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Matthew Davis| 36|  Male| 67000|
|    Patricia White| 29|Female| 50000|
|     Daniel Miller| 34|  Male| 64000|
| Elizabeth Jackson| 30|Female| 52000|
|     Joseph Harris| 28|  Male| 53000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+

+----------+
|avg_salary|
+----------+
|   57200.0|
+----------+



# Managing temporary views

In [7]:
# Check if a temporary view exists
if spark.catalog._jcatalog.tableExists('persons'):
    print('The temporary view persons exists')

# Drop the temporary view
spark.catalog.dropTempView('persons')

# Check if a temporary view exists
if spark.catalog._jcatalog.tableExists('persons'):
    print('The temporary view persons exists')

The temporary view persons exists


# Sub Queries

In [8]:
# Create two DataFrames
# The first DataFrame contains employee data with columns: id, name
# The second DataFrame contains salary data with columns: id, salary, department
data1 = [(1, 'John'), (2, 'Jane'), (3, 'Alice')]
data2 = [(1, 1000, 'HR'), (2, 1500, 'Engineering'), (3, 1200, 'Marketing')]
columns1 = ['id', 'name']
columns2 = ['id', 'salary', 'department']
df1 = spark.createDataFrame(data1, columns1)
df2 = spark.createDataFrame(data2, columns2)

# Show the first DataFrame
df1.show()

# Show the second DataFrame
df2.show()

+---+-----+
| id| name|
+---+-----+
|  1| John|
|  2| Jane|
|  3|Alice|
+---+-----+

+---+------+-----------+
| id|salary| department|
+---+------+-----------+
|  1|  1000|         HR|
|  2|  1500|Engineering|
|  3|  1200|  Marketing|
+---+------+-----------+



In [9]:
# Register as temporary views
df1.createOrReplaceTempView('employees')
df2.createOrReplaceTempView('salaries')

In [10]:
# Subquery to find employees with salaries above average
query = '''
SELECT e.name, s.salary
FROM employees e
JOIN salaries s
ON e.id = s.id
WHERE s.salary > (SELECT AVG(salary) FROM salaries)
'''
result = spark.sql(query)
result.show()

+----+------+
|name|salary|
+----+------+
|Jane|  1500|
+----+------+



24/08/08 07:00:07 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2284267 ms exceeds timeout 120000 ms
24/08/08 07:00:07 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/08 07:00:07 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.